<a href="https://colab.research.google.com/github/TNN-A/us-ie-big-data-technologies/blob/master/pyspark_structured_api.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Welcome to the Colab Spark Tutorial.

We will be using Spark a few times in this course, and the _colab_ environment provides the compute (for 12 hours at a time) we need, along with this wonderful web-based notebook.

Today we will be configuring PySpark and exploring the SparkSQL features in relation to the Spark API

Source material includes [[1](https://opensource.com/article/19/3/apache-spark-and-dataframes-tutorial)]

Sections:

 1. Configuring your _colab_
 2. Using PySpark


Firstly, we need to configure the _colab_ instance

In [ ]:
!lsb_release -a

No LSB modules are available.
Distributor ID:	Ubuntu
Description:	Ubuntu 22.04.3 LTS
Release:	22.04
Codename:	jammy


In [ ]:
!apt-get update

Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:3 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:4 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Hit:6 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Ign:7 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Get:9 https://r2u.stat.illinois.edu/ubuntu jammy Release [5,713 B]
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:11 https://r2u.stat.illinois.edu/ubuntu jammy Release.gpg [793 B]
Get:12 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:13 http://security.ubuntu.com/ubuntu jammy-security/restricted amd64 Packages [3,030 kB]
Get:14 http://archive

In [ ]:
# Install java
!apt-get install openjdk-8-jdk-headless -qq > /dev/null


In [ ]:
# get spark
VERSION='3.5.2'
!wget https://dlcdn.apache.org/spark/spark-3.5.2/spark-3.5.2-bin-hadoop3.tgz

--2024-09-13 17:32:09--  https://dlcdn.apache.org/spark/spark-3.5.2/spark-3.5.2-bin-hadoop3.tgz
Resolving dlcdn.apache.org (dlcdn.apache.org)... 151.101.2.132, 2a04:4e42::644
Connecting to dlcdn.apache.org (dlcdn.apache.org)|151.101.2.132|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 400822099 (382M) [application/x-gzip]
Saving to: ‘spark-3.5.2-bin-hadoop3.tgz’

spark-3.5.2-bin-had 100%[===================>] 382.25M  37.6MB/s    in 6.0s    

2024-09-13 17:32:27 (63.4 MB/s) - ‘spark-3.5.2-bin-hadoop3.tgz’ saved [400822099/400822099]



In [ ]:
# decompress spark
!tar xf spark-3.5.2-bin-hadoop3.tgz

# install python package to help with system paths
!pip install -q findspark


In [ ]:
# Let Colab know where the java and spark folders are

import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/spark-3.5.2-bin-hadoop3"

In [ ]:
# add pyspark to sys.path using findspark
import findspark
findspark.init()

# get a spark session
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()

In [ ]:
spark

In [ ]:
# NYC Crash dataset
! curl -o rows.csv https://data.cityofnewyork.us/api/views/h9gi-nx95/rows.csv

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  427M    0  427M    0     0  3962k      0 --:--:--  0:01:50 --:--:-- 2889k


In [ ]:
import pandas as pd
nyc_df = pd.read_csv('rows.csv')

<ipython-input-48-a70f860f02a9>:2: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  nyc_df = pd.read_csv('rows.csv')


In [ ]:
# number or rows

print(len(nyc_df))

# this is quite large so we will work with a sample while we experiment in pandas as least.

2118226


In [ ]:
nyc_small = nyc_df.sample(frac=0.2, replace=False, random_state=1)

In [ ]:
# we are also going to limit the columns to those we are going to work with

nyc_small = nyc_small[['CRASH DATE', 'CONTRIBUTING FACTOR VEHICLE 1',
                       'BOROUGH', 'VEHICLE TYPE CODE 1',
                       'NUMBER OF PERSONS INJURED']]

In [ ]:
nyc_small.head(2)

,CRASH DATE,CONTRIBUTING FACTOR VEHICLE 1,BOROUGH,VEHICLE TYPE CODE 1,NUMBER OF PERSONS INJURED
430841,05/05/2019,Unsafe Speed,STATEN ISLAND,Station Wagon/Sport Utility Vehicle,0.0
1779911,04/15/2013,Other Electronic Device,QUEENS,PASSENGER VEHICLE,1.0


Let's download some url data ("Anonymized 120-day subset of the ICML-09 URL data containing 2.4 million examples and 3.2 million features" [UCI](https://archive.ics.uci.edu/ml/datasets/URL+Reputation)

In [ ]:
! wget http://kdd.ics.uci.edu/databases/kddcup99/kddcup.data_10_percent.gz
! wget http://kdd.ics.uci.edu/databases/kddcup99/kddcup.names

--2024-09-13 18:03:18--  http://kdd.ics.uci.edu/databases/kddcup99/kddcup.data_10_percent.gz
Resolving kdd.ics.uci.edu (kdd.ics.uci.edu)... 128.195.1.86
Connecting to kdd.ics.uci.edu (kdd.ics.uci.edu)|128.195.1.86|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2144903 (2.0M) [application/x-gzip]
Saving to: ‘kddcup.data_10_percent.gz’

kddcup.data_10_perc 100%[===================>]   2.04M  1.93MB/s    in 1.1s    

2024-09-13 18:03:20 (1.93 MB/s) - ‘kddcup.data_10_percent.gz’ saved [2144903/2144903]

--2024-09-13 18:03:20--  http://kdd.ics.uci.edu/databases/kddcup99/kddcup.names
Resolving kdd.ics.uci.edu (kdd.ics.uci.edu)... 128.195.1.86
Connecting to kdd.ics.uci.edu (kdd.ics.uci.edu)|128.195.1.86|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1307 (1.3K)
Saving to: ‘kddcup.names’

kddcup.names        100%[===================>]   1.28K  --.-KB/s    in 0s      

2024-09-13 18:03:20 (137 MB/s) - ‘kddcup.names’ saved [1307/1307]



In [ ]:
!gunzip kddcup.data_10_percent.gz

In [ ]:
import pandas as pd
df = pd.read_csv('kddcup.data_10_percent', header=None)

In [ ]:
df[2].value_counts()

,count
2,
ecr_i,281400
private,110893
http,64293
smtp,9723
other,7237
...,...
X11,11
tim_i,7
pm_dump,1


In [ ]:
from pyspark.sql import SQLContext


# there are nan's in the frame with strings, and spark can't 'infer the schema', so we have to help it out
# by replacing them with empty strings and forcing the column to be a string

sdf_small = SQLContext(spark).createDataFrame(nyc_small.fillna('').astype('str'))


/content/spark-3.5.2-bin-hadoop3/python/pyspark/sql/context.py:113: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


In [ ]:
raw_rdd = spark.sparkContext.textFile('kddcup.data_10_percent').cache()
raw_rdd.take(5)

['0,tcp,http,SF,181,5450,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,8,8,0.00,0.00,0.00,0.00,1.00,0.00,0.00,9,9,1.00,0.00,0.11,0.00,0.00,0.00,0.00,0.00,normal.',
 '0,tcp,http,SF,239,486,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,8,8,0.00,0.00,0.00,0.00,1.00,0.00,0.00,19,19,1.00,0.00,0.05,0.00,0.00,0.00,0.00,0.00,normal.',
 '0,tcp,http,SF,235,1337,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,8,8,0.00,0.00,0.00,0.00,1.00,0.00,0.00,29,29,1.00,0.00,0.03,0.00,0.00,0.00,0.00,0.00,normal.',
 '0,tcp,http,SF,219,1337,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,6,6,0.00,0.00,0.00,0.00,1.00,0.00,0.00,39,39,1.00,0.00,0.03,0.00,0.00,0.00,0.00,0.00,normal.',
 '0,tcp,http,SF,217,2032,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,6,6,0.00,0.00,0.00,0.00,1.00,0.00,0.00,49,49,1.00,0.00,0.02,0.00,0.00,0.00,0.00,0.00,normal.']

In [ ]:
print(sdf_small.schema)

StructType([StructField('CRASH DATE', StringType(), True), StructField('CONTRIBUTING FACTOR VEHICLE 1', StringType(), True), StructField('BOROUGH', StringType(), True), StructField('VEHICLE TYPE CODE 1', StringType(), True), StructField('NUMBER OF PERSONS INJURED', StringType(), True)])


In [ ]:
# Pandas
nyc_small.groupby('CRASH DATE')['CRASH DATE'].count().sort_values(ascending=False).head(5)

,CRASH DATE
CRASH DATE,
01/21/2014,246
02/03/2014,212
01/18/2015,209
05/19/2017,199
12/15/2017,198


In [ ]:
#spark
sdf_small.groupBy('CRASH DATE').count().orderBy('count', ascending=False).show(5)

+----------+-----+
|CRASH DATE|count|
+----------+-----+
|01/21/2014|  246|
|02/03/2014|  212|
|01/18/2015|  209|
|05/19/2017|  199|
|11/15/2018|  198|
+----------+-----+
only showing top 5 rows



In [ ]:
#pandas
nyc_small.groupby('BOROUGH')['BOROUGH'].count().sort_values(ascending=False).head(5)

,BOROUGH
BOROUGH,
BROOKLYN,92818
QUEENS,78333
MANHATTAN,65046
BRONX,43378
STATEN ISLAND,12270


In [ ]:
#spark
sdf_small.groupBy('BOROUGH').count().orderBy('count', ascending=False).show(5)

+---------+------+
|  BOROUGH| count|
+---------+------+
|         |131800|
| BROOKLYN| 92818|
|   QUEENS| 78333|
|MANHATTAN| 65046|
|    BRONX| 43378|
+---------+------+
only showing top 5 rows



In [ ]:
#pandas

nyc_small[(nyc_small.BOROUGH == 'QUEENS')]['CONTRIBUTING FACTOR VEHICLE 1'].value_counts()

# you can also use a group by (to avoid the pandas value_counts function)

nyc_small[(nyc_small.BOROUGH == 'QUEENS')].groupby(
    'CONTRIBUTING FACTOR VEHICLE 1'
)['CONTRIBUTING FACTOR VEHICLE 1'].count().sort_values(ascending=False).head(5)

,CONTRIBUTING FACTOR VEHICLE 1
CONTRIBUTING FACTOR VEHICLE 1,
Unspecified,27394
Driver Inattention/Distraction,16605
Failure to Yield Right-of-Way,6682
Backing Unsafely,3853
Following Too Closely,2804


In [ ]:
##spark

sdf_small.filter(sdf_small.BOROUGH == 'QUEENS').groupBy('CONTRIBUTING FACTOR VEHICLE 1').count().orderBy('count', ascending=False).show(5)

+-----------------------------+-----+
|CONTRIBUTING FACTOR VEHICLE 1|count|
+-----------------------------+-----+
|                  Unspecified|27394|
|         Driver Inattentio...|16605|
|         Failure to Yield ...| 6682|
|             Backing Unsafely| 3853|
|         Following Too Clo...| 2804|
+-----------------------------+-----+
only showing top 5 rows



In [ ]:
#pandas

nyc_small.groupby(['VEHICLE TYPE CODE 1', 'BOROUGH'])['NUMBER OF PERSONS INJURED'].mean().sort_values(ascending=False).head(3)

,,NUMBER OF PERSONS INJURED
VEHICLE TYPE CODE 1,BOROUGH,
bobct,BROOKLYN,5.0
PICKUP,BRONX,4.0
BACKH,QUEENS,4.0


In [ ]:
#spark


sdf_small.groupBy('VEHICLE TYPE CODE 1', 'BOROUGH').agg({'NUMBER OF PERSONS INJURED': 'mean'}).orderBy('avg(NUMBER OF PERSONS INJURED)', ascending=False).show(10)


+-------------------+---------+------------------------------+
|VEHICLE TYPE CODE 1|  BOROUGH|avg(NUMBER OF PERSONS INJURED)|
+-------------------+---------+------------------------------+
|         School bus|         |                           6.0|
|            mta bus|         |                           6.0|
|         DELIVERY T|         |                           6.0|
|              bobct| BROOKLYN|                           5.0|
|                amb|MANHATTAN|                           4.0|
|              BACKH|   QUEENS|                           4.0|
|             PICKUP|    BRONX|                           4.0|
|         FDNY LADDE|    BRONX|                           3.0|
|              HEAVY|         |                           3.0|
|             U-Haul|         |                           3.0|
+-------------------+---------+------------------------------+
only showing top 10 rows



In [ ]:
csv_rdd = raw_rdd.map(lambda row: row.split(","))
print(csv_rdd.take(2))
print(type(csv_rdd))

[['0', 'tcp', 'http', 'SF', '181', '5450', '0', '0', '0', '0', '0', '1', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '8', '8', '0.00', '0.00', '0.00', '0.00', '1.00', '0.00', '0.00', '9', '9', '1.00', '0.00', '0.11', '0.00', '0.00', '0.00', '0.00', '0.00', 'normal.'], ['0', 'tcp', 'http', 'SF', '239', '486', '0', '0', '0', '0', '0', '1', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '8', '8', '0.00', '0.00', '0.00', '0.00', '1.00', '0.00', '0.00', '19', '19', '1.00', '0.00', '0.05', '0.00', '0.00', '0.00', '0.00', '0.00', 'normal.']]
<class 'pyspark.rdd.PipelinedRDD'>


Read the csv directly into a spark dataframe by defining a schema

In [ ]:
from pyspark.sql import Row

parsed_rdd = csv_rdd.map(lambda r: Row(
    duration=int(r[0]),
    protocol_type=r[1],
    service=r[2],
    flag=r[3],
    src_bytes=int(r[4]),
    dst_bytes=int(r[5]),
    wrong_fragment=int(r[7]),
    urgent=int(r[8]),
    hot=int(r[9]),
    num_failed_logins=int(r[10]),
    num_compromised=int(r[12]),
    su_attempted=r[14],
    num_root=int(r[15]),
    num_file_creations=int(r[16]),
    label=r[-1]
    )
)
parsed_rdd.take(5)

[Row(duration=0, protocol_type='tcp', service='http', flag='SF', src_bytes=181, dst_bytes=5450, wrong_fragment=0, urgent=0, hot=0, num_failed_logins=0, num_compromised=0, su_attempted='0', num_root=0, num_file_creations=0, label='normal.'),
 Row(duration=0, protocol_type='tcp', service='http', flag='SF', src_bytes=239, dst_bytes=486, wrong_fragment=0, urgent=0, hot=0, num_failed_logins=0, num_compromised=0, su_attempted='0', num_root=0, num_file_creations=0, label='normal.'),
 Row(duration=0, protocol_type='tcp', service='http', flag='SF', src_bytes=235, dst_bytes=1337, wrong_fragment=0, urgent=0, hot=0, num_failed_logins=0, num_compromised=0, su_attempted='0', num_root=0, num_file_creations=0, label='normal.'),
 Row(duration=0, protocol_type='tcp', service='http', flag='SF', src_bytes=219, dst_bytes=1337, wrong_fragment=0, urgent=0, hot=0, num_failed_logins=0, num_compromised=0, su_attempted='0', num_root=0, num_file_creations=0, label='normal.'),
 Row(duration=0, protocol_type='tcp',

Convert the RDD to a dataframe

In [ ]:
df = spark.createDataFrame(parsed_rdd)
df.show()

+--------+-------------+-------+----+---------+---------+--------------+------+---+-----------------+---------------+------------+--------+------------------+-------+
|duration|protocol_type|service|flag|src_bytes|dst_bytes|wrong_fragment|urgent|hot|num_failed_logins|num_compromised|su_attempted|num_root|num_file_creations|  label|
+--------+-------------+-------+----+---------+---------+--------------+------+---+-----------------+---------------+------------+--------+------------------+-------+
|       0|          tcp|   http|  SF|      181|     5450|             0|     0|  0|                0|              0|           0|       0|                 0|normal.|
|       0|          tcp|   http|  SF|      239|      486|             0|     0|  0|                0|              0|           0|       0|                 0|normal.|
|       0|          tcp|   http|  SF|      235|     1337|             0|     0|  0|                0|              0|           0|       0|                 0|normal.

In [ ]:
from pyspark.sql import functions as f

In [ ]:
# register a temporary table to query against.
df.registerTempTable('data')

/content/spark-3.5.2-bin-hadoop3/python/pyspark/sql/dataframe.py:329: FutureWarning: Deprecated in 2.0, use createOrReplaceTempView instead.
  warnings.warn("Deprecated in 2.0, use createOrReplaceTempView instead.", FutureWarning)


# Comparing SQL to API

---
#0. Select columns

In [ ]:
select = spark.sql("""SELECT protocol_type, service
                      FROM data""")

In [ ]:
select.show(10)

+-------------+-------+
|protocol_type|service|
+-------------+-------+
|          tcp|   http|
|          tcp|   http|
|          tcp|   http|
|          tcp|   http|
|          tcp|   http|
|          tcp|   http|
|          tcp|   http|
|          tcp|   http|
|          tcp|   http|
|          tcp|   http|
+-------------+-------+
only showing top 10 rows



In [ ]:
select_spark = df.select('protocol_type', 'service')

In [ ]:
select_spark.show(10)

#### OR using a list also works

In [ ]:
select_spark = df.select(['protocol_type', 'service'])
select_spark.show(10)

+-------------+-------+
|protocol_type|service|
+-------------+-------+
|          tcp|   http|
|          tcp|   http|
|          tcp|   http|
|          tcp|   http|
|          tcp|   http|
|          tcp|   http|
|          tcp|   http|
|          tcp|   http|
|          tcp|   http|
|          tcp|   http|
+-------------+-------+
only showing top 10 rows



---
# 1. select as alias

In [ ]:
alias = spark.sql("""SELECT protocol_type,
                            label as flag
                     FROM data
                  """)

In [ ]:
alias.show()

In [ ]:
alias_spark = df.select('protocol_type', 'label').withColumnRenamed('label', 'flag')

In [ ]:
alias_spark.show()

+-------------+-------+
|protocol_type|   flag|
+-------------+-------+
|          tcp|normal.|
|          tcp|normal.|
|          tcp|normal.|
|          tcp|normal.|
|          tcp|normal.|
|          tcp|normal.|
|          tcp|normal.|
|          tcp|normal.|
|          tcp|normal.|
|          tcp|normal.|
|          tcp|normal.|
|          tcp|normal.|
|          tcp|normal.|
|          tcp|normal.|
|          tcp|normal.|
|          tcp|normal.|
|          tcp|normal.|
|          tcp|normal.|
|          tcp|normal.|
|          tcp|normal.|
+-------------+-------+
only showing top 20 rows



#### OR using dataframe column-objects with .alias

In [ ]:
alias_spark = df.select(df.protocol_type, df.label.alias('flag'))
alias_spark.show(10)

# 2. group by, count, order by

In [ ]:
protocols = spark.sql("""
      SELECT protocol_type, count(*) as freq
      FROM data
      GROUP BY protocol_type
      ORDER BY 2 DESC
                           """)
protocols.show()

In [ ]:
df.groupBy('protocol_type').count().orderBy('count', ascending=False).show()

In [ ]:
df.count()

---
# 3. group by, count, order by (using agg)

In [ ]:
labels = spark.sql("""
  SELECT label, count(*) as freq
  FROM data
  GROUP BY label
  ORDER BY 2 DESC
""")

In [ ]:
labels.show()

In [ ]:
labels_spark = df.groupBy('label')\
                .agg(f.count(f.lit(1))\
                    .alias('freq'))\
                    .orderBy('freq', ascending=False)

In [ ]:
labels_spark.show()

---
#4. case, group by, count, order by

In [ ]:
attack_protocol = spark.sql("""
                           SELECT
                             protocol_type,
                             CASE label
                               WHEN 'normal.' THEN 'no attack'
                               ELSE 'attack'
                             END AS state,
                             COUNT(*) as freq
                           FROM data
                           GROUP BY protocol_type, state
                           ORDER BY 3 DESC
                           """)

In [ ]:
attack_protocol.show()

In [ ]:
att_prot_spark = df.withColumn('state', f.when(df.label=='normal.', 'no attack').otherwise('attack'))\
                  .groupBy('protocol_type', 'state')\
                  .agg(f.count(f.lit(1)).alias('freq'))\
                  .orderBy('freq', ascending=False)


In [ ]:
att_prot_spark.show()

---
#5. group by, aggregations

In [ ]:
attack_stats = spark.sql("""
                          SELECT
                            protocol_type,
                            CASE label
                              WHEN 'normal.' THEN 'no attack'
                              ELSE 'attack'
                            END AS state,
                            COUNT(*) as total_freq,
                            ROUND(AVG(src_bytes), 2) as mean_src_bytes,
                            ROUND(AVG(dst_bytes), 2) as mean_dst_bytes,
                            ROUND(AVG(duration), 2) as mean_duration,
                            SUM(num_failed_logins) as total_failed_logins,
                            SUM(num_compromised) as total_compromised,
                            SUM(num_file_creations) as total_file_creations,
                            SUM(su_attempted) as total_root_attempts,
                            SUM(num_root) as total_root_acceses
                          FROM data
                          GROUP BY protocol_type, state
                          ORDER BY 3 DESC
                          """)

In [ ]:
attack_stats.show()

In [ ]:
attack_stats_spark = df.withColumn('state', f.when(df.label=='normal.', 'no attack').otherwise('attack'))\
.groupBy('protocol_type', 'state')\
.agg(f.count(f.lit(1)).alias('total_freq'),
     f.avg('src_bytes').alias('mean_src_bytes'),
     f.avg('dst_bytes').alias('mean_dst_bytes'),
     f.avg('duration').alias('mean_duration'),
     f.sum('num_failed_logins').alias('total_failed_logins'),
     f.sum('num_compromised').alias('total_compromised'),
     f.sum('num_file_creations').alias('total_file_creations'),
     f.sum('su_attempted').alias('total_root_attempts'),
     f.sum('num_root').alias('total_root_acceses'),
     )\
     .orderBy('total_freq', ascending=False)

In [ ]:
attack_stats_spark.show()

---
# 6. filter, group by

In [ ]:
tcp_attack_stats = spark.sql("""
                              SELECT
                                service,
                                label as attack_type,
                                COUNT(*) as total_freq,
                                ROUND(AVG(duration), 2) as mean_duration,
                                SUM(num_failed_logins) as total_failed_logins,
                                SUM(num_file_creations) as total_file_creations,
                                SUM(su_attempted) as total_root_attempts,
                                SUM(num_root) as total_root_acceses
                              FROM data
                              WHERE protocol_type = 'tcp'
                              AND label != 'normal.'
                              GROUP BY service, attack_type
                              ORDER BY total_freq DESC
                              """)

In [ ]:
tcp_attack_stats.show()

In [ ]:
tcp_attack_stats_spark = df.filter((df.protocol_type  == "tcp") & (df.label  != "normal.")).groupBy('service', df.label.alias('attack_type'))\
.agg(f.count(f.lit(1)).alias('total_freq'),
     f.avg('duration').alias('mean_duration'),
     f.sum('num_failed_logins').alias('total_failed_logins'),
     f.sum('num_file_creations').alias('total_file_creations'),
     f.sum('su_attempted').alias('total_root_attempts'),
     f.sum('num_root').alias('total_root_acceses'))\
.orderBy('total_freq', ascending=False)

In [ ]:
tcp_attack_stats_spark.show()

---
#7. sub-queries

In [ ]:
tcp_attack_stats = spark.sql("""
                              SELECT
                                t.service,
                                t.attack_type,
                                t.total_freq
                              FROM
                              (SELECT
                                service,
                                label as attack_type,
                                COUNT(*) as total_freq,
                                ROUND(AVG(duration), 2) as mean_duration,
                                SUM(num_failed_logins) as total_failed_logins,
                                SUM(num_file_creations) as total_file_creations,
                                SUM(su_attempted) as total_root_attempts,
                                SUM(num_root) as total_root_acceses
                              FROM data
                              WHERE protocol_type = 'tcp'
                              AND label != 'normal.'
                              GROUP BY service, attack_type
                              ORDER BY total_freq DESC) as t
                                WHERE t.mean_duration > 0
                              """)

In [ ]:
tcp_attack_stats.show()

In [ ]:
tcp_attack_stats_spark = df.filter((df.protocol_type  == "tcp") & (df.label  != "normal."))\
.groupBy('service', df.label.alias('attack_type'))\
.agg(f.count(f.lit(1)).alias('total_freq'),
     f.avg('duration').alias('mean_duration'),
     f.sum('num_failed_logins').alias('total_failed_logins'),
     f.sum('num_file_creations').alias('total_file_creations'),
     f.sum('su_attempted').alias('total_root_attempts'),
     f.sum('num_root').alias('total_root_acceses'))\
.orderBy('total_freq', ascending=False)\
.filter(f.col('mean_duration') > 0)\
.select('service', 'attack_type', 'total_freq')

In [ ]:
tcp_attack_stats_spark.show()